# PyTorch implementation 



### Variables and basics

In [31]:
import numpy as np
import torch

In [5]:
t1 = torch.tensor(4.)
t2 = torch.tensor([1.,2.,3,4])
print(t1)
print(t1.dtype)
print(t2)
print(t2.dtype)

In [7]:
t3 = torch.tensor([[1.,2.],[3,4],[5,6]])
print(t3.dtype)
print(t3.shape)

#### Operations and gradients

In [20]:
x = torch.tensor(3.)
w = torch.tensor(4.,requires_grad = True)
b = torch.tensor(5., requires_grad = True)

In [21]:
y = w*x + b
y

In [22]:
y.backward()       


In [23]:
print('dy/dx', x.grad)
print('dy/dw', w.grad)
print('dy/db', b.grad)

#### Tensor functions

In [27]:
t6 = torch.full([6,3],10)
t7 = torch.full([4,3],12)
print(t6)
print(t7)
t8 = torch.cat((t6,t7))
t8

#### Sine of each element of an array
#### Reshaping

In [30]:
t9 = torch.sin(t8)
print(t9)
t10 = t9.reshape((5,6))
print(t10)

#### Interpolabilty with numpy

In [37]:
x = np.array([[1,2],[3,4.]])
print(x)
y = torch.from_numpy(x)
print(y)
print(x.dtype,y.dtype)
z = y.numpy()
print(z.dtype)

## Gradient Descent 

In [73]:
import torch
import numpy as np

#### Creating the training data (simple matrix)

In [74]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [75]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

#### Initializing the weights and bias for the linear regression model

In [76]:
w = torch.randn(2,3, requires_grad = True)
b = torch.randn(2,requires_grad = True)
print(w)
print(b)

In [77]:
def model(x):
    return x@w.t() + b # Returns the matrix multiplication of x with w transpose, adds b by broadcasting

In [78]:
preds = model(inputs)
print(preds)
print(targets)

In [79]:
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/ diff.numel()
    

In [80]:
loss = mse(preds,targets)
print(loss)

In [81]:
loss.backward()
print(w)
print(w.grad)

In [82]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [83]:
loss = mse(preds , targets)
loss

In [84]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)
print(w)

#### Training the network for multiple epochs using the helper functions 

In [90]:
for i in range(10000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
    if i%100 == 0:
        print("Loss after iteration {} is {}",i,loss)

In [91]:
preds = model(inputs)
preds

In [93]:
error = abs(preds - targets)

#### Linear regression using inbuilt Torch functionalities


In [94]:
import torch.nn as nn


In [95]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [96]:
from torch.utils.data import TensorDataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]


In [98]:
from torch.utils.data import DataLoader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [99]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

In [101]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)
list(model.parameters())

In [102]:
preds = model(inputs)
preds

In [103]:
import torch.nn.functional as F

In [104]:
loss_fn = F.mse_loss

In [105]:
loss = loss_fn(model(inputs), targets)
print(loss)

In [106]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [107]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [109]:
fit(1000, model, loss_fn, opt, train_dl)

In [112]:
preds = model(inputs)
print(preds)
print(targets)